# Коллекции и с чем их едят.

Мы уже знакомы с несколькими видами коллекций - списками (`list`), кортежами (`tuple`), строками (`str`), диапазонами (`range`).

Сегодня мы познакомимся еще с двумя видами - множеством (`set`) и словарём (`dict`). 

Если к этому списку добавить коллекции из стандартной библиотеки (например, `deque`, `defaultdict` и `OrderedDict`), а еще и популярные коллекции из других библиотек (вспомнить тот же `NumPy`), то кажется, что познакомиться со всем этим разнообразием - непосильная задача. Благо сам `Python` устанавливает некоторую иерархию среди всех коллекций и для понимания бывает полезно разобраться в ней.

### Примечание. 

Дальнейшие определения основаны на "утиной типизации" (`duck typing` - https://en.wikipedia.org/wiki/Duck_typing), которую `Python` успешно поддерживает. 

В программировании это приложение "утиного теста" (duck test):
 > If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck.

См. документацию: https://docs.python.org/3/glossary.html#term-duck-typing

### Контейнер (`Container`).

Объект считается контейнером (`Container`), если у него можно спросить, содержит ли он произвольный элемент `x`. 

*Обычно*, для проверки этого мы можем использовать оператор `in`:

Если `x in A` исполняется корректно и возвращает значение типа `bool`, то `A` - *обычно* является контейнером.

 > Вообще говоря, это не совсем так. 
 >
 > Для любого *итерируемого* объекта мы тоже можем использовать `in` корректно, но не все итерируемые объекты являются контейнерами. Важная разница между контейнерами и некоторыми итерируемыми объектами заключается в том, что при итерациях контейнеры возвращают существующие значения, а генераторы и файлы (`file`) создают новые объекты каждый раз. 
 >
 > Необходимое условие для того, чтобы назвать объект контейнером - у него должен быть перегружен метод `__contains__` (именно это и позволяет использовать `in` слева от этого объекта)
 >
 > Это означает, что условие `isinstance(A, collections.abc.Container)` гарантирует нам, что `A` поддерживает оператор `in`.
 > 
 > Подробнее см. https://stackoverflow.com/a/11576019
 
Пример, показывающий, что объект, явно не являющийся контейнером, может поддерживать оператор `in`:

In [58]:
import collections
import random

x = iter(lambda: random.choice(range(6)), 0)
print(isinstance(x, collections.abc.Container))
print(2 in x)


False
True


### Итерируемый объект (`Iterable`).

Понятие итерируемости тесно связано с итераторами, но про них разговор пройдет сильно позже.

Объект считается итерируемым (`iterable`), если:

 - **По нему можно пробежаться в цикле (`iterate over`) - если `for x in A: ...` может быть корректно исполнено, значит `A` - итерируемый объект.**

 - К нему можно применить `iter()` и вернётся *итератор*.
    
 - У него есть метод `__iter__`, возвращающий новый *итератор* или у него есть метод `__getitem__` для обращения по индексу.

 Документация: https://docs.python.org/dev/glossary.html#term-iterable

 Подробнее про итерируемость, итераторы и итерации: https://stackoverflow.com/questions/9884132/what-are-iterator-iterable-and-iteration


### Объект ограниченной длины (`Sized`).

Объект считается `Sized`, если у него можно спросить количество элементов функцией `len()`.

Немного конкретнее, если `len(A)` выполняется без ошибок и возвращает целое неотрицательное число, то `A` - `Sized` объект.

`len(A)` корректно выполняется, если у класса, к которому относится `A` (пере)определен магический метод `__len__`

Теперь можем перейти к сути разговора - к коллекциям.

## Коллекции (`Collections`).

Коллекция - объект, который одновременно является *контейнером*, *итерируемым объектом* и *объектом ограниченной длинны*.

Разработчики языка python смотрят на эту иерархию понятий как на иерархию классов, где абстрактный класс `Collection` наследуется от всех трех выше обозначенных абстрактных классов: `Container`, `Iterable` и `Sized`.

![Картинка](https://fadeevlecturer.github.io/python_lectures/_images/collections_venn.svg)
![Alt text](https://fadeevlecturer.github.io/python_lectures/_images/collections_hierarchy.svg)

In [59]:
from collections.abc import Collection, Container, Iterable, Sized

print(issubclass(Collection, Container))
print(issubclass(Collection, Iterable))
print(issubclass(Collection, Sized))


True
True
True


![Alt text](https://fadeevlecturer.github.io/python_lectures/_images/sequence_vs_mapping.svg)

#### А дальше что?

Дальнейшая классификация коллекций обычно производится по способу получения доступа к элементам. 

По такому принципу предлагаю выделить три основных класса (хотя существуют не только они) — последовательности (`Sequence`), отображения (`Mapping`) и множества (`Set`).

Кроме того, все коллекции можно разделить на два вида - изменяемые (`Mutable`) и не изменяемые (`Immutable`).



## `Sequence`

Определение - https://docs.python.org/dev/glossary.html#term-sequence

**Коллекция является последовательностью, если элементы этой коллекции упорядоченны. Следствием этого свойства является возможность индексации последовательности по порядковому номеру элемента (по смещению, по целочисленному индексу), т.е. у любой последовательности можно спросить её i-й элемент.**

То, с чем мы уже привыкли работать - `list`, `str`, `tuple`, `range` - все они являются *последовательностями*.

Именно это и объясняет схожий интерфейс взаимодействия с ними (вспоминаем оператор квадратных скобок, слайсы (`slices`), конкатенацию и различные методы, которые едины для них всех (хотя `range` немного отличается, на нём не всё работает))

Всё вышеперечисленное хорошо нам знакомо, так что не будем повторяться. Для того, чтобы освежить в памяти что-либо, рекомендую:

https://fadeevlecturer.github.io/python_lectures/notebooks/python/sequences.html

## `Set` и `frozenset`

"Set" на русский язык переводится как "множество" и в `Python` по всем своим параметрам является дискретным множеством с точки зрения математики.

`frozenset` - это неизменяемая (`immutable`) версия обычных `set`'ов, так что далее речь пойдет только про обычные. Любые операции, изменяющие само множество, очевидно, работать с `frozenset` не будут, но всё остальное остается верным.

`Set` - это ***неупорядоченная*** коллекция ***различных*** хэшируемых (`hashable`) элементов. 

Неупорядоченность означает, что `set`'ы никак не запоминают порядок хранения или добавления элементов, а также не допускают обращения по индексу, слайсов и другого `sequence-like` поведения.

In [5]:
# "Хаотичный" порядок элементов
s = set()
s.add(10)
s.add(20242343234234)
s.add(3350845)
print(s)


{20242343234234, 10, 3350845}


Элементы `set` уникальны. Два одинаковых элемента не могут храниться в одном `set`.

In [18]:
# Элементы уникальны. Добавлении копии не изменяет множество.
s = {10, 20, 30, 40, 50}
s.add(10)
print(s)


{50, 20, 40, 10, 30}


### Что можно делать с множествами?

https://docs.python.org/3/library/stdtypes.html#set

#### Очевидно, создавать.

In [29]:
# Способ первый - создать пустое множество и добавить что-то впоследствии
set_first = set()
set_first.add(10)
print(set_first)


{10}


In [30]:
# Способ второй - ручками задать (заметьте, при создании можно задать несколько копий одного элемента, но в итоге в set окажется только одна копия)
set_second = {"cat", "dog", "cat", "rat", "banana", "synchrophasotron"}
print(set_second)


{'dog', 'rat', 'cat', 'banana', 'synchrophasotron'}


In [18]:
# Способ третий - использовать конструктор класса set от чего-то iterable
set_third = set("Antananarivo")
print(set_third)


{'n', 'a', 'v', 'o', 'i', 'A', 'r', 't'}


In [19]:
# Способ четвертый - set comprehention
set_fourth = {letter.lower() * 3 for letter in "Эйяфьядлайёкюдль" if letter != "ь"}
print(set_fourth)


{'яяя', 'ккк', 'ююю', 'ффф', 'ддд', 'ллл', 'ййй', 'эээ', 'ёёё', 'ааа'}


#### Изменять.

Начнем с обычных изменений:

 - `s.add(elem)` добавляет элемент в множество. Не делает ничего, если элемент с таким же значением уже есть
 - `s.pop()` удаляет и возвращает случайный элемент.
 - `s.discard(elem)` удаляет элемент со значением `elem`. Не делает ничего, если такого элемента нет. У него есть брат-близнец `remove`, который при удалении значения, не имеющегося во множестве, выдает ошибку.
 - `s.clear()` удаляет ВСЕ элементы множества.

Далее операции над множествами:

Обратите внимание, что есть два варианта использовать одни и те же операции. Разница в них лишь в том, что методы поддерживают своими аргументами любые `iterable` объекты, в отличие от операторов.

 - Пересечение 
 
       `a.intersection(b, c)` или `a & b & c`
 - Объединение 

       `a.union(b, c)` или `a | b | c`
 - Разность 

       `a.difference(b, c)` или `a - b - c`

 - Симметричная разница 

       `a.symmetric_difference(b)` или `a ^ b`



![Alt text](https://habrastorage.org/r/w1560/files/8dc/1ae/16d/8dc1ae16db9c4432938a8e79b97eefe3.png)

In [49]:
# Нахождение пересечения всех трех множеств:
a = {1, 3, 4, "z", "^_^"}
b = {3, "x", "z", "^_^"}
c = {3, "T_T", "0_0", "-_-", "^_^", ":):"}

intersection_abc = a.intersection(b, c)
print(intersection_abc)


{3, '^_^'}


In [50]:
# Нахождение объединения всех трех множеств:
a = {1, 3, 4, "z", "^_^"}
b = {3, "x", "z", "^_^"}
c = {3, "T_T", "0_0", "-_-", "^_^", ":):"}
union_abc = a.union(b, c)
print(union_abc)


{1, 'T_T', 3, 4, 'x', ':):', '-_-', '0_0', '^_^', 'z'}


In [51]:
# Нахождение разности между множеством c и двумя другими:
a = {1, 3, 4, "z", "^_^"}
b = {3, "x", "z", "^_^"}
c = {3, "T_T", "0_0", "-_-", "^_^", ":):"}
difference_abc = c.difference(a, b)
print(difference_abc)


{'T_T', ':):', '-_-', '0_0'}


In [52]:
# Нахождение симметричной разницы c и a:
a = {1, 3, 4, "z", "^_^"}
b = {3, "x", "z", "^_^"}
c = {3, "T_T", "0_0", "-_-", "^_^", ":):"}
symmetric_diff = c.symmetric_difference(a)
print(symmetric_diff)


{1, 4, ':):', '0_0', 'T_T', '-_-', 'z'}


Существуют еще `_update` версии тех же операций, они обновляют исходное множество (на который используется метод) вместо того, чтобы возвращать новое множество.

## `Dict`

Полностью копирует материал https://fadeevlecturer.github.io/python_lectures/notebooks/python/dictionaries.html, так что кому удобнее - читайте в браузере.

Документация: https://docs.python.org/3/library/stdtypes.html#mapping-types-dict

Новый тип контейнера - [dict](https://docs.python.org/3/library/stdtypes.html#mapping-types-dict) (словарь, отображение, хэш-таблица), который, как и любой другой контейнер, хранит в себе элементы, но в отличие от ранее рассмотренных контейнеров, словари **не** являются последовательностями (`sequences`), т.е. элементы словаря не упорядочены и к ним нельзя обращаться по индексу. 

Словари в `python` **изменяемы**.

### Хэш-таблицы 

По сути дела словари в `python` реализуют [хеш-таблицу](https://ru.wikipedia.org/wiki/%D0%A5%D0%B5%D1%88-%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0) --- распространенную во многих языках программирования структуру данных. Наиболее близкий аналог к словарям из `python` в `C/C++` --- [std::unordered_map](https://en.cppreference.com/w/cpp/container/unordered_map). Хеш-таблицы хранят пары (`ключ`, `значение`) (`key`, `value`) и позволяют выполнять следующие три операции:
1) добавлять элемент по ключу;
2) удалять элемент по ключу;
3) искать элемент по ключу.

При хорошей реализации все эти три операции работают очень быстро и даже более того, скорость их работы практически не зависит от количества хранимых элементов. В `python` словари являются чуть ли не самой ключевой структурой данных --- на них много завязано. Поэтому они сильно оптимизированы под скорость, но за это приходиться платить сравнительно большими накладными расходами по памяти. 


### `Hash`

Работа хеш-таблицы опирается на хеш-функцию. 

*Хеш-функция* — функция, которая на вход принимает произвольный ключ и возвращает целое число в диапазоне $[0, ..., M]$:

$ h: K \rightarrow \lfloor 0, ..., M\rfloor $

где $K$ - множество ключей, $M$ - максимальное значение хеш-функции

Пусть $(k, v)$ — пара (ключ, значение). Идея заключается в том, чтобы хранить все такие пары в обычном массиве $A$ размера $M+1$, где индекс ячейки массива для данной пары $(k, v)$ определяется значением хеш-функции $h(k)$ от ключа $k$:

$A \lfloor h(k)\rfloor = (k, v)$

Такая конструкция позволяет искать пару (ключ, значение) по ключу за время индексации массива $A$ плюс время вычисления хеш-функции $h(k)$. Индексация сплошного массива по смещению — очень быстрая операция, а скорость вычисления значения хеш-функции — одно из ключевых требований к хорошей хеш-функции.

В реальности может найтись два таких ключа $k_1$ и $k_2$, что значение хеш-функций на них совпадут: $h(k_1) = h(k_2)$. Такое явление называют *коллизией*, для разрешения которых разработано множество методов. Время поиска ключа в таблице замедляется, если встречаются коллизии. В связи с этим минимизация количества коллизий — ещё одно ключевое требование к хорошим хеш-функциям.


 - хеш-таблицы хранят пары (ключ, значение);

 - для их работы необходима возможность вычислять значение хеш-функции от ключей, т.е. ключи должны быть хэшируемы (`hashable`);

 - кроме того, ключи в хеш-таблице должны быть уникальными;

 - хеш-таблицы позволяют очень быстро искать, добавлять и удалять пары (ключ, значение) по ключу.

### Создание словарей

Как и в случае всех предыдущих контейнеров, создавать словари можно множеством разных способов.
- Используя фигурные скобки "`{}`" и помещая внутри пары `ключ`: `значение`,  отделяя ключ от значения символом двоеточия "`:`", и разделяя пары друг от друга запятыми "`,`":

In [20]:
my_first_dict = {
    "language": "python", 
    "version": 3.11, 
    "room": 309
}
print(my_first_dict)


{'language': 'python', 'version': 3.11, 'room': 309}


- Используя конструктор [dict](https://docs.python.org/3/library/functions.html#func-dict):

In [54]:
a_dict_from_iterable = dict(
    [("foo", 100), ("bar", 200)]
)  # словарь из списка пар (key, value)
a_dict_from_kwargs = dict(foo=100, bar=200)  # используя именованные аргументы
print(a_dict_from_iterable, a_dict_from_kwargs)


{'foo': 100, 'bar': 200} {'foo': 100, 'bar': 200}


- пустой словарь можно создать, используя пустую пару фигурных скобок "`{}`" или ничего не передав конструктору `dict`:

In [55]:
an_empty_dict = dict()
another_empty_dict = {}
print(an_empty_dict, another_empty_dict)


{} {}


- Используя `dict comprehensions`: 

In [56]:
squares = {x: x**2 for x in range(10)}
print(squares)


{0: 0, 1: 1, 2: 4, 3: 9, 4: 16, 5: 25, 6: 36, 7: 49, 8: 64, 9: 81}


### Добавление, поиск и удаление по ключу в `dict`

Добавление, удаление и поиск значений по ключу разберем реализовав словарем ... англо-русский словарь названий цифр. Ниже создаётся такой словарь. 

In [22]:
eng_to_ru = {
    "one": "один",
    "two": "два",
    "three": "три",
    "fou": "четыре",
    "five": "пят",
    "six": "шесть",
    "seven": "семь",
    "eight": "восемь",
    "nine": "девять",
}
print(eng_to_ru)


{'one': 'один', 'two': 'два', 'three': 'три', 'fou': 'четыре', 'five': 'пят', 'six': 'шесть', 'seven': 'семь', 'eight': 'восемь', 'nine': 'девять'}


Английское название цифры выступает в качестве ключа в нашем словаре, а его перевод выступает в качестве значения. Это позволяет нам по английскому варианту (ключу) быстро найти русский вариант (значение). 

Если `d` --- словарь, а `key` --- ключ, то для получения значения по этому ключу используется синтаксис
```python
d[key]
```

In [59]:
print(eng_to_ru["one"])
print(eng_to_ru["seven"])


один
семь


Код выше находит русскоязычные варианты слов "one" и "seven" в словаре, передавая их в качестве ключа.

Искать таким синтаксисом можно только по существующим ключам. Например, попробуем спросить у словаря перевод слова "four".

In [60]:
print(eng_to_ru["four"])


KeyError: 'four'

Возникла ошибка [KeyError](https://docs.python.org/3/library/exceptions.html#KeyError), сигнализирующая об отсутствии переданного ключа в словаре. Она возникла, т.к. при заполнении словаря была совершена опечатка: вместо ключа "four" был введен ключ "fou". 

Эту ошибку можно исправить, т.к. словари изменяемы:
- в словаре можно изменять значение для уже существующего **ключа**; 
- добавлять новую пару (`ключ`, `значение`);
- удалять пару (`ключ`, `значение`) по **ключу**. 

Используем эти возможности для исправления опечаток в словаре. Для начала обратим внимание, что по ключу "five" находится значение "пят", а не "пять".

In [61]:
print(eng_to_ru["five"])


пят


Чтобы изменить значение по ключу, достаточно присвоить по этому ключу новое значение, т.е. если `d` --- словарь, `key` --- ключ, по которому требуется заменить старое значение на новое значение `new_value`, то используется синтаксис
```python
d[key] = new_value
```

Воспользуемся этим синтаксисом, чтобы исправить опечатку по ключу "five".

In [62]:
eng_to_ru["five"] = "пять"
print(eng_to_ru)


{'one': 'один', 'two': 'два', 'three': 'три', 'fou': 'четыре', 'five': 'пять', 'six': 'шесть', 'seven': 'семь', 'eight': 'восемь', 'nine': 'девять'}


Видим, что значение по ключу "five" удалось успешно изменить на "пять". Осталась ещё опечатка в ключе `four`. 

Словари не предусматривают операции редактирования ключа напрямую. Тем не менее можно добиться схожего эффекта за два шага:
1. удалить пару (`ключ`, `значение`) по неверному ключу.
2. добавить пару (`ключ`, `значение`) по исправленному ключу.

```{note}
На ключи намеренно накладывают требование неизменяемости, чтобы исключить возможности ключа изменения ключа в словаре по разделяемой ссылке.
```

Чтобы удалить пару (`ключ`, `значение`) из словаря `d` по ключу `key` используется синтаксис
```python
del d[key]
```

Применим этот синтаксис для того, чтобы удалить ключ `fou` из словаря. Здесь удобно также продемонстрировать, что хоть словари и не являются последовательностью, но их длину, т.е. количество ключей (или количество значение) можно спросить функцией `len`.

In [63]:
print(len(eng_to_ru))
del eng_to_ru["fou"]
print(len((eng_to_ru)))
print(eng_to_ru)


9
8
{'one': 'один', 'two': 'два', 'three': 'три', 'five': 'пять', 'six': 'шесть', 'seven': 'семь', 'eight': 'восемь', 'nine': 'девять'}


По изменению количества элементов и по содержимому словаря видно, что успешно удалось удалить пару ("fou", "четыре").

Чтобы добавить в словарь значение по новому ключу используется тот же синтаксис, что и для изменения значения по уже существующему ключу, т.е. если `d` --- словарь и требуется добавить пару (`new_key`, `new_value`), то используется синтаксис
```python
d[new_key] = new_value
```
И теперь добавим значение `"четыре"` по верному ключу `"four"`.

In [64]:
eng_to_ru["four"] = "четыре"
print(eng_to_ru)


{'one': 'один', 'two': 'два', 'three': 'три', 'five': 'пять', 'six': 'шесть', 'seven': 'семь', 'eight': 'восемь', 'nine': 'девять', 'four': 'четыре'}


## Методы словаря

Как и у списков, у словарей есть множество методов для работы с ними. Во-первых, можно проверять наличие ключа словаря, тем же синтаксисом, что проверяется наличие элемента в последовательностях, т.е. вычисление выражения
```python
key in d
```
вернет `True`, если в словаре `d` есть ключ `key`, и значение `False` иначе.

In [65]:
print("one" in eng_to_ru)


True


Однако если вам хочется проверить наличие ключа, только потому что вы не уверены, что такой ключ будет присутствовать в словаре и избегаете возбуждение исключения `KeyError`, то лучше использовать метод [get](https://docs.python.org/3/library/stdtypes.html#dict.get), который возвращает значение по ключу, если таковой присутствует и возвращает заданное значение по умолчанию, если ключ отсутствует. 

In [66]:
print(eng_to_ru.get("one", "Перевод не известен!"))
print(eng_to_ru.get("eleven", "Перевод не известен!"))


один
Перевод не известен!


Есть очень похожий метод [pop](https://docs.python.org/3/library/stdtypes.html#dict.pop), который работает точно также, но если ключ в словаре присутствует, то пара извлекается из словаря и вызывающему коду возвращается значение из этой пары.

In [67]:
eng_to_ru["zero"] = "ноль"
print(eng_to_ru)
print(eng_to_ru.pop("zero", "Перевод не известен!"))
print(eng_to_ru)


{'one': 'один', 'two': 'два', 'three': 'три', 'five': 'пять', 'six': 'шесть', 'seven': 'семь', 'eight': 'восемь', 'nine': 'девять', 'four': 'четыре', 'zero': 'ноль'}
ноль
{'one': 'один', 'two': 'два', 'three': 'три', 'five': 'пять', 'six': 'шесть', 'seven': 'семь', 'eight': 'восемь', 'nine': 'девять', 'four': 'четыре'}


### Итерация по словарю

По словарю можно итерироваться разными способами. Если указать в цикле `for` сам словарь, то итерация будет производиться по его ключам.

In [23]:
for _ in eng_to_ru:
    print(_, end=" ")


one two three fou five six seven eight nine 

Если требуются не ключи, а только значения, то удобно использовать метод [values](https://docs.python.org/3/library/stdtypes.html#dict.values).

In [69]:
for value in eng_to_ru.values():
    print(value, end=" ")


один два три пять шесть семь восемь девять четыре 

Если требуются и ключи и значения, то оптимальнее всего воспользоваться методом [items](https://docs.python.org/3/library/stdtypes.html#dict.values), который итерируется сразу по парам (`ключ`, `значение`).

In [70]:
for key, value in eng_to_ru.items():
    print(f"{key:6} => {value}")


one    => один
two    => два
three  => три
five   => пять
six    => шесть
seven  => семь
eight  => восемь
nine   => девять
four   => четыре


В ходе написания всего этого, были использованы материалы (aka "читайте там подробнее"):

 - Скопипастил всё про словари у замечательного автора: https://fadeevlecturer.github.io/python_lectures/notebooks/python/dictionaries.html
 
